In [ ]:
Greta Samuitytė 2110618
Modelis VGG16
Klasės: balloon, dog, car
greta.samuityte@mif.stud.vu.lt

In [9]:
import torch
from torchvision import datasets
from torchvision.transforms import transforms
import numpy as np
from torchvision import models

In [10]:
%pip install openimages
import os
from openimages.download import download_dataset

data_dir = "data"
number_for_samples = 334
classes = ["Balloon", "Dog", "Car"]

if not os.path.exists(data_dir):
  os.makedirs(data_dir)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.5 MB/s eta 0:00:00


In [11]:
download_dataset(data_dir, classes, limit=number_for_samples)

100%|██████████| 334/334 [00:08<00:00, 38.58it/s]


{'balloon': {'images_dir': 'data/balloon/images'},
 'dog': {'images_dir': 'data/dog/images'},
 'car': {'images_dir': 'data/car/images'}}

In [20]:
from torchvision import transforms

# Normalization parameters
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Enhanced transformation pipeline with additional augmentations
all_transforms = transforms.Compose([
    transforms.Resize(256),  # Resize the shortest side to 256 pixels
    transforms.CenterCrop(224),  # Crop the center 224x224 pixels of the image
    transforms.RandomHorizontalFlip(),  # Random horizontal flipping
    transforms.RandomVerticalFlip(),  # Random vertical flipping (for variety)
    transforms.RandomRotation(degrees=15),  # Random rotation by 15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color jittering
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),  # Random affine transformations
    transforms.ToTensor(),  # Convert the image to a tensor
    normalize  # Normalize the image
])


In [21]:
dataset = datasets.ImageFolder(root='data', transform=all_transforms)

In [22]:
batch_holder = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2)

In [23]:
torch.cuda.is_available()

True

In [16]:
model = models.vgg16(pretrained = True)
model.to("cuda")
model.eval()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 98.1MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [17]:
def calculate_metrics(ground_truth, predictions, threshold = 0.9):
  predictions_thresholded = (predictions > threshold).astype(np.float64)
  TP = np.sum(np.bitwise_and(ground_truth == 1, predictions_thresholded == 1))
  TN = np.sum(np.bitwise_and(ground_truth == 0, predictions_thresholded == 0))
  FP = np.sum(np.bitwise_and(ground_truth == 0, predictions_thresholded == 1))
  FN = np.sum(np.bitwise_and(ground_truth == 1, predictions_thresholded == 0))
  metrics = {}
  metrics['accuracy'] = (TP + TN) / (TP + FP + TN + FN)
  metrics['recall'] = TP / (TP + FN)
  metrics['precision'] = TP / (TP + FP)
  metrics['f1'] = 2 * (metrics['precision'] * metrics['recall']) / (metrics['precision'] + metrics['recall'])
  return metrics

In [25]:
# tikimybių traukimas iš visų klasių masyvo ir sudėjimas į tokio tipo sarašą: [(tikimybe1, tiesa2),(tikimybė2, tiesa2), ...]
def calculate_prob(arr2D, class_id, label_arr, class_label):
  result = []
  label_bool = 1.0
  for i in range(0, len(label_arr)):
    if class_label == label_arr[i]:
      label_bool = 1.0
    else:
      label_bool = 0.0
    result.append((arr2D[i][class_id], label_bool))
  return result

# tikimybių traukimas iš praeitoje funkcijoje sudaryto sąrašo. gauname tikimybių sąrašą: [tikimybė1, tikimybė2, ...]
def extract_prob(arr):
  result = []
  for prob, truth in arr:
    result.append(prob)
  return result

# tiesų traukimas iš pirmoje funkcijoje sudaryto sąrašo. gauname tikimybių sąrašą: [tiesa1, tiesa2, ...]
def extract_gt(arr):
  result = []
  for prob, truth in arr:
    result.append(truth)
  return result

def extractAndCalculate(gt_prob_list, Threshold_list):
    best_metrics = {'threshold': 0, 'accuracy': 0, 'recall': 0, 'precision': 0, 'f1': 0}

    for threshold in Threshold_list:
        gt_list = extract_gt(gt_prob_list)
        pred_list = extract_prob(gt_prob_list)
        metrics = calculate_metrics(np.array(gt_list), np.array(pred_list), threshold)

        if metrics['f1'] > best_metrics['f1']:  # Update best_metrics if current F1 is better
            best_metrics = metrics
            best_metrics['threshold'] = threshold

    print(f"Best Threshold: {best_metrics['threshold']}, Best F1: {best_metrics['f1']}")
    return best_metrics


In [26]:
probabilities_list = []
labels_list = []

ballon_class_id = 417
dog_class_id = 953
car_class_id = 340


Threshold_list = np.linspace(0, 0.99, 21)
b_accuracies, b_precisions, b_recalls, b_f1s = [], [], [], []
d_accuracies, d_precisions, d_recalls, d_f1s = [], [], [], []
c_accuracies, c_precisions, c_recalls, c_f1s = [], [], [], []


for threshold in Threshold_list:
#treshold = 0.90
  ballon_truth_probablity_list = []
  dog_truth_probablity_list = []
  car_truth_probablity_list = []


  for images, labels in batch_holder:
    images, labels = images.cuda(), labels.cuda()
    outputs = model(images)
    probabilities = torch.sigmoid(outputs).detach().cpu().numpy()
    labels_array = labels.cpu().numpy()

    ballon_truth_probablity_list.extend(calculate_prob(probabilities, ballon_class_id, labels_array, 0))
    dog_truth_probablity_list.extend(calculate_prob(probabilities, dog_class_id, labels_array, 1))
    car_truth_probablity_list.extend(calculate_prob(probabilities, car_class_id, labels_array, 2))

  b_res = extractAndCalculate(ballon_truth_probablity_list, Threshold_list)
  b_accuracies.append(b_res['accuracy'])
  b_recalls.append(b_res['recall'])
  b_precisions.append(b_res['precision'])
  b_f1s.append(b_res['f1'])


  d_res = extractAndCalculate(dog_truth_probablity_list, Threshold_list)
  d_accuracies.append(d_res['accuracy'])
  d_recalls.append(d_res['recall'])
  d_precisions.append(d_res['precision'])
  d_f1s.append(d_res['f1'])

  c_res = extractAndCalculate(car_truth_probablity_list, Threshold_list)
  c_accuracies.append(c_res['accuracy'])
  c_recalls.append(c_res['recall'])
  c_precisions.append(c_res['precision'])
  c_f1s.append(c_res['f1'])

b_threshold_index = np.argmax(b_f1s)
d_threshold_index = np.argmax(d_f1s)
c_threshold_index = np.argmax(c_f1s)


print(f"Ballon best threshold: {Threshold_list[b_threshold_index]} accuracy: {b_accuracies[b_threshold_index]} recall: {b_recalls[b_threshold_index]} precision: {b_precisions[b_threshold_index]} f1: {b_f1s[b_threshold_index]}")
print(f"Dog best threshold: {Threshold_list[d_threshold_index]} accuracy: {d_accuracies[d_threshold_index]} recall: {d_recalls[d_threshold_index]} precision: {d_precisions[d_threshold_index]} f1: {d_f1s[d_threshold_index]}")
print(f"Car best threshold: {Threshold_list[c_threshold_index]} accuracy: {c_accuracies[c_threshold_index]} recall: {c_recalls[c_threshold_index]} precision: {c_precisions[c_threshold_index]} f1: {c_f1s[c_threshold_index]}")



Best Threshold: 0.99, Best F1: 0.7658227848101267
Best Threshold: 0.0, Best F1: 0.5
Best Threshold: 0.0, Best F1: 0.5
Best Threshold: 0.99, Best F1: 0.7725856697819314
Best Threshold: 0.0, Best F1: 0.5
Best Threshold: 0.0495, Best F1: 0.5020441537203597
Best Threshold: 0.99, Best F1: 0.761904761904762
Best Threshold: 0.0, Best F1: 0.5
Best Threshold: 0.0495, Best F1: 0.500408830744072
Best Threshold: 0.99, Best F1: 0.7767295597484276
Best Threshold: 0.0, Best F1: 0.5
Best Threshold: 0.0495, Best F1: 0.5012126111560226
Best Threshold: 0.99, Best F1: 0.7737909516380655
Best Threshold: 0.0, Best F1: 0.5
Best Threshold: 0.0, Best F1: 0.5
Best Threshold: 0.99, Best F1: 0.7605633802816901
Best Threshold: 0.0, Best F1: 0.5
Best Threshold: 0.099, Best F1: 0.50355871886121
Best Threshold: 0.99, Best F1: 0.7599364069952305
Best Threshold: 0.0, Best F1: 0.5
Best Threshold: 0.0495, Best F1: 0.5094185094185094
Best Threshold: 0.99, Best F1: 0.7731629392971245
Best Threshold: 0.0, Best F1: 0.5
Best 